# **Fine-tuning Wav2Vec2 for Vietnamese ASR with 🤗(huggingface) Transformers**

Wav2Vec2 is a pretrained model for Automatic Speech Recognition (ASR) and was released in [September 2020](https://ai.facebook.com/blog/wav2vec-20-learning-the-structure-of-speech-from-raw-audio/) by Alexei Baevski, Michael Auli, and Alex Conneau.

Using a novel contrastive pretraining objective, Wav2Vec2 learns powerful speech representations from more than 50.000 hours of unlabeled speech. Similar, to [BERT's masked language modeling](http://jalammar.github.io/illustrated-bert/), the model learns contextualized speech representations by randomly masking feature vectors before passing them to a transformer network.

![wav2vec2_structure](https://raw.githubusercontent.com/patrickvonplaten/scientific_images/master/wav2vec2.png)

For the first time, it has been shown that pretraining, followed by fine-tuning on very little labeled speech data achieves competitive results to state-of-the-art ASR systems. Using as little as 10 minutes of labeled data, Wav2Vec2 yields a word error rate (WER) of less than 5% on the clean test set of [LibriSpeech](https://huggingface.co/datasets/librispeech_asr) - *cf.* with Table 9 of the [paper](https://arxiv.org/pdf/2006.11477.pdf).

In this notebook, we will give an in-detail explanation of how Wav2Vec2's pretrained checkpoints can be fine-tuned on any English ASR dataset. Note that in this notebook, we will fine-tune Wav2Vec2 without making use of a language model. It is much simpler to use Wav2Vec2 without a language model as an end-to-end ASR system and it has been shown that a standalone Wav2Vec2 acoustic model achieves impressive results. For demonstration purposes, we fine-tune the "base"-sized [pretrained checkpoint](https://huggingface.co/facebook/wav2vec2-base) on the rather small [Timit](https://huggingface.co/datasets/timit_asr) dataset that contains just 5h of training data.

Wav2Vec2 is fine-tuned using Connectionist Temporal Classification (CTC), which is an algorithm that is used to train neural networks for sequence-to-sequence problems and mainly in Automatic Speech Recognition and handwriting recognition.

I highly recommend reading the blog post [Sequence Modeling with CTC (2017)](https://distill.pub/2017/ctc/) very well-written blog post by Awni Hannun.

First, let's try to get a good GPU in our colab! With Google Colab's free version it's sadly becoming much harder to get access to a good GPU. With Google Colab Pro, one has a much easier time getting access to a V100 or P100 GPU however.

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Thu Dec 28 10:03:59 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

Before we start, let's install both `datasets` and `transformers` from master. Also, we need the `librosa` package to load audio files and the `jiwer` to evaluate our fine-tuned model using the [word error rate (WER)](https://huggingface.co/metrics/wer) metric ${}^1$.

In [ ]:
#%%capture
# !pip install datasets==1.18.3
# !pip install transformers==4.17.0
# !pip install jiwer
#mấy cái này rất hay lỗi phiên bản nên chú ý phiên bản
%%capture
!pip install datasets==1.18.3
!pip install transformers
!pip install jiwer
!pip install huggingface_hub==0.20.1
!pip install accelerate==0.25.0

In [ ]:
import huggingface_hub
huggingface_hub.__version__
#0.19.4->0.20.1

'0.20.1'

In [ ]:
# prompt: kiểm tra phiên bản tranformer
import transformers
transformers.__version__
#4.17.0->4.36.2

'4.36.2'

In [ ]:
!pip install --upgrade huggingface_hub


In [ ]:
# prompt: tranformers update

!pip install --upgrade transformers


In [ ]:
!pip install accelerate -U

Next we strongly suggest to upload your training checkpoints directly to the [🤗 Hub](https://huggingface.co/) while training. The [🤗 Hub](https://huggingface.co/) has integrated version control so you can be sure that no model checkpoint is getting lost during training.

To do so you have to store your authentication token from the Hugging Face website (sign up [here](https://huggingface.co/join) if you haven't already!)

In [ ]:
from huggingface_hub import notebook_login

notebook_login()


Then you need to install Git-LFS to upload your model checkpoints:

In [ ]:
%%capture
!apt install git-lfs




---

${}^1$ Timit is usually evaluated using the phoneme error rate (PER), but by far the most common metric in ASR is the word error rate (WER). To keep this notebook as general as possible we decided to evaluate the model using WER.

## Prepare Data, Tokenizer, Feature Extractor

ASR models transcribe speech to text, which means that we both need a feature extractor that processes the speech signal to the model's input format, *e.g.* a feature vector, and a tokenizer that processes the model's output format to text.

In 🤗 Transformers, the Wav2Vec2 model is thus accompanied by both a tokenizer, called [Wav2Vec2CTCTokenizer](https://huggingface.co/transformers/master/model_doc/wav2vec2.html#wav2vec2ctctokenizer), and a feature extractor, called [Wav2Vec2FeatureExtractor](https://huggingface.co/transformers/master/model_doc/wav2vec2.html#wav2vec2featureextractor).

Let's start by creating the tokenizer responsible for decoding the model's predictions.

### Create Wav2Vec2CTCTokenizer

The [pretrained Wav2Vec2 checkpoint]( ) maps the speech signal to a sequence of context representations as illustrated in the figure above. A fine-tuned Wav2Vec2 checkpoint needs to map this sequence of context representations to its corresponding transcription so that a linear layer has to be added on top of the transformer block (shown in yellow). This linear layer is used to classifies each context representation to a token class analogous how, *e.g.*, after pretraining a linear layer is added on top of BERT's embeddings for further classification - *cf.* with *"BERT"* section of this [blog post](https://huggingface.co/blog/warm-starting-encoder-decoder).

The output size of this layer corresponds to the number of tokens in the vocabulary, which does **not** depend on Wav2Vec2's pretraining task, but only on the labeled dataset used for fine-tuning. So in the first step, we will take a look at Timit and define a vocabulary based on the dataset's transcriptions.

Let's start by loading the dataset and taking a look at its structure.

In [ ]:
# prompt: mã kết nối drive

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!pwd
!mkdir Data

/content


In [ ]:
#!cp -r /content/drive/MyDrive/ColabNotebooks_2/CS337_XULI_AMTHANH_TIENGNOI/DATA/train_data.zip /content/Data
%cd /content/Data
!unzip /content/drive/MyDrive/ColabNotebooks_2/CS337_XULI_AMTHANH_TIENGNOI/DATA/train_data.zip

!cp -r /content/drive/MyDrive/ColabNotebooks_2/CS337_XULI_AMTHANH_TIENGNOI/DATA/train_final_20230919.jsonl /content/Data

Kết quả truyền trực tuyến bị cắt bớt đến 5000 dòng cuối.
  inflating: Train/64888019629cf7532b239f22.wav  
  inflating: Train/64889b82629cf79aef23b642.wav  
  inflating: Train/648c85faf73a57a6b67d4c84.wav  
  inflating: Train/6485cbff8503510057e4cdec.wav  
  inflating: Train/648d3882ca834ba9b0159d57.wav  
  inflating: Train/648dc44c4c84025d9f3e0824.wav  
  inflating: Train/648c0c50f7c92ce17a4e8e3a.wav  
  inflating: Train/648725bd1ae6766ba5b659bf.wav  
  inflating: Train/64832ef3d9f569017c41f6a7.wav  
  inflating: Train/6485cde1850351144be4d0a6.wav  
  inflating: Train/6485c933850351f504e4ca8c.wav  
  inflating: Train/648b26e7608325b646e93d88.wav  
  inflating: Train/648adca8b7c4444cd2394eb6.wav  
  inflating: Train/648dd0e04c84028a413e1b93.wav  
  inflating: Train/64833383d9f569d72741f816.wav  
  inflating: Train/64871d601ae6763489b64be2.wav  
  inflating: Train/648b191060832571f3e91bed.wav  
  inflating: Train/648dd75d4c840219703e27d6.wav  
  inflating: Train/648c8091f73a576ed97d393e

In [ ]:
!ls /content/Data/Train | wc -l

7490


In [ ]:
from IPython.lib.display import Audio
import torchaudio
import torch
from datasets import load_dataset, Dataset, DatasetDict, concatenate_datasets,load_metric
import json
import pandas as pd
import os

In [ ]:
data = []
with open("/content/Data/train_final_20230919.jsonl") as f:
    for line in f:
        data.append(json.loads(line))

df = pd.DataFrame(data)
data = Dataset.from_pandas(df)

In [ ]:
data

Dataset({
    features: ['id', 'sentence', 'intent', 'sentence_annotation', 'entities', 'file'],
    num_rows: 7490
})

Let's write a short function to display some random samples of the dataset and run it a couple of times to get a feeling for the transcriptions.

In [ ]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(data, num_examples=5)

,id,sentence,intent,sentence_annotation,entities,file
0,648f30bdbd5f0190f6bbfd3e,giảm giúp mình cái bóng ốp trần xuống 73% nhé,giảm độ sáng của thiết bị,[ command : giảm ] giúp mình cái [ device : bóng ốp trần ] xuống [ target number : 73% ] nhé,"[{'filler': 'giảm', 'type': 'command'}, {'filler': 'bóng ốp trần', 'type': 'device'}, {'filler': '73%', 'type': 'target number'}]",648f30bdbd5f0190f6bbfd3e.wav
1,648b1dff6083257c5ae925ea,bạn ơi bật giúp mình cái máy in với,bật thiết bị,bạn ơi [ command : bật ] giúp mình cái [ device : máy in ] với,"[{'filler': 'bật', 'type': 'command'}, {'filler': 'máy in', 'type': 'device'}]",648b1dff6083257c5ae925ea.wav
2,64873b431ae67617d2b680e3,đóng giúp tôi cái vòi tưới với,đóng thiết bị,[ command : đóng ] giúp tôi cái [ device : vòi tưới ] với,"[{'filler': 'đóng', 'type': 'command'}, {'filler': 'vòi tưới', 'type': 'device'}]",64873b431ae67617d2b680e3.wav
3,6489c81c000750733a9236f4,"bạn ơi, lúc 3 giờ 10 phút bạn bật cho mình cái bóng ngủ nhớ",bật thiết bị,"bạn ơi, lúc [ time at : 3 giờ 10 phút ]bạn [ command : bật ] cho mình cái[ device : bóng ngủ ] nhớ","[{'filler': '3 giờ 10 phút', 'type': 'time at'}, {'filler': 'bật', 'type': 'command'}, {'filler': 'bóng ngủ', 'type': 'device'}]",6489c81c000750733a9236f4.wav
4,648c78d6f73a5797997d1e10,trời sáng quá đóng cho anh cái rèm ở phòng chính vào đi,đóng thiết bị,trời sáng quá [ command : đóng ] cho anh cái [ device : rèm ] ở [ location : phòng chính ] vào đi,"[{'filler': 'đóng', 'type': 'command'}, {'filler': 'rèm', 'type': 'device'}, {'filler': 'phòng chính', 'type': 'location'}]",648c78d6f73a5797997d1e10.wav


Chúng ta chỉ sử dụng cột sentence và file nên xóa các cột khác đi

In [ ]:
data=data.remove_columns(["id","intent","sentence_annotation","entities"])

In [ ]:
data

Dataset({
    features: ['sentence', 'file'],
    num_rows: 7490
})

In [ ]:
data[10]

{'sentence': 'kiểm tra xem cái đèn học tắt chưa nhé',
 'file': '64831bb0d9f56988a941ef80.wav'}

Alright! The transcriptions look very clean and the language seems to correspond more to written text than dialogue. This makes sense taking into account that [Timit](https://huggingface.co/datasets/timit_asr) is a read speech corpus.

We can see that the transcriptions contain some special characters, such as `,.?!;:`. Without a language model, it is much harder to classify speech chunks to such special characters because they don't really correspond to a characteristic sound unit. *E.g.*, the letter `"s"` has a more or less clear sound, whereas the special character `"."` does not.
Also in order to understand the meaning of a speech signal, it is usually not necessary to include special characters in the transcription.

In addition, we normalize the text to only have lower case letters and append a word separator token at the end.

In [ ]:
# prompt: viết mã tìm tất cả các kí tự đặc biệt trong cột sentence

# import re
# special_char=[]
# for i in range(len(data)):
#   sentence = data[i]["sentence"]
#   sentence = re.sub('[^a-z]', '', sentence)



In [ ]:
import re
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\\/"]'

def remove_special_characters(batch):
    batch["text"] = re.sub(chars_to_ignore_regex, '', batch["sentence"]).lower() + " "
    return batch

In [ ]:
data = data.map(remove_special_characters)

0ex [00:00, ?ex/s]

In [ ]:
data

Dataset({
    features: ['sentence', 'file', 'text'],
    num_rows: 7490
})

In [ ]:
show_random_elements(data)

,sentence,file,text
0,thôi chết rồi giảm giúp mình cái quạt thông gió xuống với,648b298e60832567b3e94546.wav,thôi chết rồi giảm giúp mình cái quạt thông gió xuống với
1,tăng đèn lên 47%,648729f91ae67681b0b66159.wav,tăng đèn lên 47%
2,đóng giúp chị cái lò nướng ở trong nhà tắm,648c8ed8f73a5711e07d6a4c.wav,đóng giúp chị cái lò nướng ở trong nhà tắm
3,kiểm tra bóng trụ cổng lúc 5 giờ 29 phút cho anh,64874c651ae676a77eb696c4.wav,kiểm tra bóng trụ cổng lúc 5 giờ 29 phút cho anh
4,em ơi bật giúp anh cái đèn led,648c81a4f73a5790887d3d13.wav,em ơi bật giúp anh cái đèn led
5,bây giờ tôi đang bận nhưng vào lúc 13 giờ 43 phút mở radio cho tôi,648bde82f7c92c2c224e8d68.wav,bây giờ tôi đang bận nhưng vào lúc 13 giờ 43 phút mở radio cho tôi
6,giảm cái đèn tuýp giúp chị với,648737591ae6765b73b67a5f.wav,giảm cái đèn tuýp giúp chị với
7,bạn ơi giảm cho mình cái bóng hắt trong phòng thu với sáng quá,648b3d5b60832511b4e97676.wav,bạn ơi giảm cho mình cái bóng hắt trong phòng thu với sáng quá
8,trong phòng bí quá tăng giúp mình cái quạt thông gió lên với,648b2ce66083257e66e94de7.wav,trong phòng bí quá tăng giúp mình cái quạt thông gió lên với
9,"tràn rồi, em ơi, đóng giúp anh cái van nước ở phòng tắm xông hơi với",648c84dcf73a5725c77d4859.wav,tràn rồi em ơi đóng giúp anh cái van nước ở phòng tắm xông hơi với


Good! This looks better. We have removed most special characters from transcriptions and normalized them to lower-case only.

In CTC, it is common to classify speech chunks into letters, so we will do the same here.
Let's extract all distinct letters of the training and test data and build our vocabulary from this set of letters.

We write a mapping function that concatenates all transcriptions into one long transcription and then transforms the string into a set of chars.
It is important to pass the argument `batched=True` to the `map(...)` function so that the mapping function has access to all transcriptions at once.

In [ ]:
def extract_all_chars(batch):
  all_text = " ".join(batch["text"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}

In [ ]:
vocabs = data.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=data.column_names)
#ko có remove_columns=data.column_names sẽ lỗi bởi vì trả về có 1 hàng à, mà cái data cũ nó 7490 hàng nên ko gán vô dc, phải xóa cái cục 7490 đi

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
vocabs

Dataset({
    features: ['vocab', 'all_text'],
    num_rows: 1
})

In [ ]:
print(len(vocabs["vocab"][0]))
vocabs['vocab']

105


[['e',
  'ẩ',
  'ù',
  'ợ',
  'â',
  'à',
  'ở',
  'á',
  'ể',
  'ễ',
  'o',
  'ổ',
  'y',
  'ệ',
  'ụ',
  'ố',
  'ế',
  'õ',
  'ờ',
  'ỏ',
  'x',
  'z',
  'ó',
  'b',
  'ử',
  'l',
  '7',
  'ộ',
  'c',
  '3',
  'ẻ',
  'ê',
  'ầ',
  'r',
  'é',
  '̀',
  '5',
  'ì',
  'ă',
  'ủ',
  'ớ',
  'p',
  'ắ',
  'ề',
  'ặ',
  'ỳ',
  'ừ',
  'k',
  '0',
  '̣',
  'ả',
  'ú',
  '%',
  'g',
  'ỷ',
  'ọ',
  'a',
  'ồ',
  'ỡ',
  'ã',
  'ô',
  'm',
  'h',
  'ậ',
  'ạ',
  'ằ',
  'ẵ',
  'ứ',
  '4',
  'ũ',
  'ẽ',
  '2',
  'ơ',
  'ò',
  'd',
  'ẹ',
  '6',
  'đ',
  't',
  'ự',
  ' ',
  'è',
  'ữ',
  'v',
  'í',
  'ỉ',
  '9',
  '1',
  'ấ',
  '́',
  'ư',
  'w',
  'u',
  'ẫ',
  'ỗ',
  's',
  'n',
  'ẳ',
  'ý',
  'i',
  '8',
  'q',
  'f',
  'ĩ',
  'ị']]

Bộ dữ liệu khá nhỏ có thể không chứa hết chữ nên ta sẽ tải vocab từ nguyenlevubinh, sau đó lấy hợp 2 bộ vocab này lại

In [ ]:
from transformers import Wav2Vec2Processor
processor = Wav2Vec2Processor.from_pretrained("nguyenvulebinh/wav2vec2-base-vietnamese-250h")

pretrain_vocab = processor.tokenizer.get_vocab()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
len(pretrain_vocab)

112

In [ ]:
vocab_list = list(set(vocabs["vocab"][0]) |
                  set(pretrain_vocab.keys()) - {"<s>", "</s>", "<pad>", "<unk>", "|", "j"})

In [ ]:
len(vocab_list)

108

In [ ]:
vocab_dict = {k: v for v, k in enumerate(vocab_list)}
vocab_dict.keys()

dict_keys(['e', 'ẩ', 'ù', 'ợ', 'â', 'à', 'ở', 'á', 'ể', 'ễ', 'o', 'ổ', 'y', 'ệ', 'ụ', 'ố', 'ế', 'у', 'õ', 'ờ', 'ỏ', 'x', 'z', 'ó', 'b', 'ử', 'l', '7', 'ộ', 'c', '3', 'ẻ', 'ê', 'ầ', 'r', 'é', '̀', '5', 'ì', 'ă', 'ủ', 'ớ', 'p', 'ắ', 'ề', 'ặ', 'ỳ', 'ừ', 'k', '0', '̣', 'ả', 'ú', '%', 'g', 'ỹ', 'ỷ', 'ọ', 'a', 'ồ', 'ỡ', 'ã', 'ô', 'm', 'h', 'ậ', 'ạ', 'ằ', 'ẵ', 'ứ', '4', 'ũ', 'ẽ', 'ơ', '2', 'ò', 'd', 'ẹ', '6', 'đ', 't', 'ự', ' ', 'è', 'ữ', 'v', 'í', 'ỉ', '9', '1', 'ỵ', 'ấ', '́', 'ư', 'w', 'u', 'ẫ', 'ỗ', 's', 'n', 'ẳ', 'ý', 'i', '8', 'q', 'f', 'ĩ', 'ị'])

Cool, we see that all letters of the alphabet occur in the dataset (which is not really surprising) and we also extracted the special characters `" "` and `'`. Note that we did not exclude those special characters because:

- The model has to learn to predict when a word finished or else the model prediction would always be a sequence of chars which would make it impossible to separate words from each other.
- In English, we need to keep the `'` character to differentiate between words, *e.g.*, `"it's"` and `"its"` which have very different meanings.

To make it clearer that `" "` has its own token class, we give it a more visible character `|`. In addition, we also add an "unknown" token so that the model can later deal with characters not encountered in Timit's training set.

Finally, we also add a padding token that corresponds to CTC's "*blank token*". The "blank token" is a core component of the CTC algorithm. For more information, please take a look at the "Alignment" section [here](https://distill.pub/2017/ctc/).

In [ ]:
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]

In [ ]:
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
len(vocab_dict)

110

In [ ]:
vocab_dict

{'e': 0,
 'ẩ': 1,
 'ù': 2,
 'ợ': 3,
 'â': 4,
 'à': 5,
 'ở': 6,
 'á': 7,
 'ể': 8,
 'ễ': 9,
 'o': 10,
 'ổ': 11,
 'y': 12,
 'ệ': 13,
 'ụ': 14,
 'ố': 15,
 'ế': 16,
 'у': 17,
 'õ': 18,
 'ờ': 19,
 'ỏ': 20,
 'x': 21,
 'z': 22,
 'ó': 23,
 'b': 24,
 'ử': 25,
 'l': 26,
 '7': 27,
 'ộ': 28,
 'c': 29,
 '3': 30,
 'ẻ': 31,
 'ê': 32,
 'ầ': 33,
 'r': 34,
 'é': 35,
 '̀': 36,
 '5': 37,
 'ì': 38,
 'ă': 39,
 'ủ': 40,
 'ớ': 41,
 'p': 42,
 'ắ': 43,
 'ề': 44,
 'ặ': 45,
 'ỳ': 46,
 'ừ': 47,
 'k': 48,
 '0': 49,
 '̣': 50,
 'ả': 51,
 'ú': 52,
 '%': 53,
 'g': 54,
 'ỹ': 55,
 'ỷ': 56,
 'ọ': 57,
 'a': 58,
 'ồ': 59,
 'ỡ': 60,
 'ã': 61,
 'ô': 62,
 'm': 63,
 'h': 64,
 'ậ': 65,
 'ạ': 66,
 'ằ': 67,
 'ẵ': 68,
 'ứ': 69,
 '4': 70,
 'ũ': 71,
 'ẽ': 72,
 'ơ': 73,
 '2': 74,
 'ò': 75,
 'd': 76,
 'ẹ': 77,
 '6': 78,
 'đ': 79,
 't': 80,
 'ự': 81,
 'è': 83,
 'ữ': 84,
 'v': 85,
 'í': 86,
 'ỉ': 87,
 '9': 88,
 '1': 89,
 'ỵ': 90,
 'ấ': 91,
 '́': 92,
 'ư': 93,
 'w': 94,
 'u': 95,
 'ẫ': 96,
 'ỗ': 97,
 's': 98,
 'n': 99,
 'ẳ': 100,
 'ý': 101

Cool, now our vocabulary is complete and consists of 110 tokens, which means that the linear layer that we will add on top of the pretrained Wav2Vec2 checkpoint will have an output dimension of 110.

Let's now save the vocabulary as a json file.

In [ ]:
import json
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

In [ ]:
# prompt: giải thích mã này
# import json
# with open('vocab.json', 'w') as vocab_file:
#     json.dump(vocab_dict, vocab_file)

The code snippet you provided opens a file named 'vocab.json' in write mode ('w') and uses the json.dump() function to write the contents of the vocab_dict dictionary to the file.

Here's a breakdown of the code:

1. **Opening the File:**



In a final step, we use the json file to instantiate an object of the `Wav2Vec2CTCTokenizer` class.

In [ ]:
from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer("./vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

If one wants to re-use the just created tokenizer with the fine-tuned model of this notebook, it is strongly advised to upload the `tokenizer` to the [🤗 Hub](https://huggingface.co/). Let's call the repo to which we will upload the files
`"wav2vec2-base-timit-demo-colab"`:

In [ ]:
repo_name = "CS337_finetuning_wav2vecbase_vietnamese_soict23slu_nguyenlevubinh250h"

and upload the tokenizer to the [🤗 Hub](https://huggingface.co/).

In [ ]:
tokenizer.push_to_hub(repo_name)
#error: gặp lỗi tự sửa k dc thì lên gg search or hỏi chatbox --> do lỗi phiên bản
#tư duy: code này hồi xưa họ chạy dc mà giờ ko chạy dc lỗi k có đối số, khả năng cao là thư viện được cập nhật -> hãy ktr phiên bản
#TypeError: create_repo() got an unexpected keyword argument 'organization'
#ValueError: You must login to the Hugging Face hub on this computer by typing transformers-cli login
#and entering your credentials to use use_auth_token=True. Alternatively, you can pass your own token as the use_auth_token argument
#https://github.com/huggingface/transformers/issues/22621
#https://discuss.huggingface.co/t/typeerror-init-got-an-unexpected-keyword-argument-hub-token/19864


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


CommitInfo(commit_url='https://huggingface.co/QuangTuan/CS337_finetuning_wav2vecbase_vietnamese_soict23slu_nguyenlevubinh250h/commit/6d5dbba19a22433b046c6a999e04cf67fe3142ca', commit_message='Upload tokenizer', commit_description='', oid='6d5dbba19a22433b046c6a999e04cf67fe3142ca', pr_url=None, pr_revision=None, pr_num=None)

Great, you can see the just created repository under `https://huggingface.co/<your-username>/wav2vec2-base-timit-demo-colab`

In [ ]:
# prompt: tải tokenizer từ repo huggingface
from transformers import Wav2Vec2CTCTokenizer
tokenizer = Wav2Vec2CTCTokenizer.from_pretrained("QuangTuan/CS337_finetuning_wav2vecbase_vietnamese_soict23slu_nguyenlevubinh250h")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.34k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/32.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/96.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


### Create Wav2Vec2 Feature Extractor

Speech is a continuous signal and to be treated by computers, it first has to be discretized, which is usually called **sampling**. The sampling rate hereby plays an important role in that it defines how many data points of the speech signal are measured per second. Therefore, sampling with a higher sampling rate results in a better approximation of the *real* speech signal but also necessitates more values per second.

A pretrained checkpoint expects its input data to have been sampled more or less from the same distribution as the data it was trained on. The same speech signals sampled at two different rates have a very different distribution, *e.g.*, doubling the sampling rate results in data points being twice as long. Thus,
before fine-tuning a pretrained checkpoint of an ASR model, it is crucial to verify that the sampling rate of the data that was used to pretrain the model matches the sampling rate of the dataset used to fine-tune the model.

Wav2Vec2 was pretrained on the audio data of [LibriSpeech](https://huggingface.co/datasets/librispeech_asr) and LibriVox which both were sampling with 16kHz. Our fine-tuning dataset, [Timit](hhtps://huggingface.co/datasets/timit_asr), was luckily also sampled with 16kHz. If the fine-tuning dataset would have been sampled with a rate lower or higher than 16kHz, we first would have had to up or downsample the speech signal to match the sampling rate of the data used for pretraining.



A Wav2Vec2 feature extractor object requires the following parameters to be instantiated:

- `feature_size`: Speech models take a sequence of feature vectors as an input. While the length of this sequence obviously varies, the feature size should not. In the case of Wav2Vec2, the feature size is 1 because the model was trained on the raw speech signal ${}^2$.
- `sampling_rate`: The sampling rate at which the model is trained on.
- `padding_value`: For batched inference, shorter inputs need to be padded with a specific value
- `do_normalize`: Whether the input should be *zero-mean-unit-variance* normalized or not. Usually, speech models perform better when normalizing the input
- `return_attention_mask`: Whether the model should make use of an `attention_mask` for batched inference. In general, models should **always** make use of the `attention_mask` to mask padded tokens. However, due to a very specific design choice of `Wav2Vec2`'s "base" checkpoint, better results are achieved when using no `attention_mask`. This is **not** recommended for other speech models. For more information, one can take a look at [this](https://github.com/pytorch/fairseq/issues/3227) issue. **Important** If you want to use this notebook to fine-tune [large-lv60](https://huggingface.co/facebook/wav2vec2-large-lv60), this parameter should be set to `True`.

In [ ]:
from transformers import Wav2Vec2FeatureExtractor

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=False)

Great, Wav2Vec2's feature extraction pipeline is thereby fully defined!

To make the usage of Wav2Vec2 as user-friendly as possible, the feature extractor and tokenizer are *wrapped* into a single `Wav2Vec2Processor` class so that one only needs a `model` and `processor` object.

In [ ]:
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [ ]:
#processor.push_to_hub(repo_name)

Next, we can prepare the dataset.

### Preprocess Data


In [ ]:
import soundfile as sf
# để hiểu 2 cells này thì ta chạy code đã, đọc code chỗ nào k biết ta in ra
def speech_file_to_array_fn(batch):# để hiểu hàm này thì ở đây ta đặt câu hỏi batch là gì
    #nếu k biết thì ta in nó ra
    #print(batch)
    #print(1)# in ra số 1 để biết cái batch dc in ra đến đâu
    #nó là 1 hàng data
    speech_array, sampling_rate = sf.read('/content/Data/Train/'+batch["file"])
    batch["speech"] = speech_array
    batch["sampling_rate"] = sampling_rate
    batch["target_text"] = batch["text"]
    return batch

In [ ]:
data = data.map(speech_file_to_array_fn, remove_columns=data.column_names)
#hàm map có chức năng cho từng hàng vào hàm ở trên,hàm ở trên trả về từng hàng mới có thêm 3 thuộc tính
#ta xóa 3 thuộc tính cũ đi, xếp chồng các hàng trả về này lại lưu vào biến data
data

#đoạn này data đã được đưa về dạng giống trong blog. có 3 cột

0ex [00:00, ?ex/s]

Dataset({
    features: ['speech', 'sampling_rate', 'target_text'],
    num_rows: 7490
})

In [ ]:
data[0]

{'speech': [0.0,
  -3.0517578125e-05,
  0.0,
  -3.0517578125e-05,
  0.0,
  -3.0517578125e-05,
  0.0,
  -3.0517578125e-05,
  0.0,
  -3.0517578125e-05,
  0.0,
  -3.0517578125e-05,
  0.0,
  0.0,
  0.0,
  -3.0517578125e-05,
  -3.0517578125e-05,
  -3.0517578125e-05,
  0.0,
  -3.0517578125e-05,
  -3.0517578125e-05,
  0.0,
  0.0,
  0.0,
  0.0,
  -3.0517578125e-05,
  -3.0517578125e-05,
  -3.0517578125e-05,
  0.0,
  -3.0517578125e-05,
  0.0,
  0.0,
  0.0,
  3.0517578125e-05,
  3.0517578125e-05,
  -6.103515625e-05,
  -3.0517578125e-05,
  0.0,
  -3.0517578125e-05,
  -3.0517578125e-05,
  -3.0517578125e-05,
  -3.0517578125e-05,
  -3.0517578125e-05,
  -3.0517578125e-05,
  -3.0517578125e-05,
  -3.0517578125e-05,
  -3.0517578125e-05,
  -3.0517578125e-05,
  -3.0517578125e-05,
  -3.0517578125e-05,
  -3.0517578125e-05,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  -3.0517578125e-05,
  -3.0517578125e-05,
  0.0,
  -3.0517578125e-05,
  -3.0517578125e-05,
  -3.05175781

We can see that the audio file has automatically been loaded. This is thanks to the new [`"Audio"` feature](https://huggingface.co/docs/datasets/package_reference/main_classes.html?highlight=audio#datasets.Audio) introduced in `datasets == 4.13.3`, which loads and resamples audio files on-the-fly upon calling.

The sampling rate is set to 16kHz which is what `Wav2Vec2` expects as an input.

Great, let's listen to a couple of audio files to better understand the dataset and verify that the audio was correctly loaded.

**Note**: *You can click the following cell a couple of times to listen to different speech samples.*

In [ ]:
import IPython.display as ipd
import numpy as np
import random

rand_int = random.randint(0, len(data))

print(data[rand_int]["target_text"])
ipd.Audio(data=np.asarray(data[rand_int]["speech"]), autoplay=True, rate=16000)

It can be heard, that the speakers change along with their speaking rate, accent, etc. Overall, the recordings sound relatively clear though, which is to be expected from a read speech corpus.

Let's do a final check that the data is correctly prepared, by printing the shape of the speech input, its transcription, and the corresponding sampling rate.

**Note**: *You can click the following cell a couple of times to verify multiple samples.*

In [ ]:
rand_int = random.randint(0, len(data))

print("Target text:", data[rand_int]["target_text"])
print("Input array shape:", np.asarray(data[rand_int]["speech"]).shape)#nhìn kĩ tránh nhầm
print("Sampling rate:", data[rand_int]["sampling_rate"])

Target text: trời nóng quá đóng giúp mình cái rèm nhé 
Input array shape: (86400,)
Sampling rate: 16000


Good! Everything looks fine - the data is a 1-dimensional array, the sampling rate always corresponds to 16kHz, and the target text is normalized.

Finally, we can process the dataset to the format expected by the model for training. We will make use of the `map(...)` function.

First, we load and resample the audio data, simply by calling `batch["audio"]`.
Second, we extract the `input_values` from the loaded audio file. In our case, the `Wav2Vec2Processor` only normalizes the data. For other speech models, however, this step can include more complex feature extraction, such as [Log-Mel feature extraction](https://en.wikipedia.org/wiki/Mel-frequency_cepstrum).
Third, we encode the transcriptions to label ids.

**Note**: This mapping function is a good example of how the `Wav2Vec2Processor` class should be used. In "normal" context, calling `processor(...)` is redirected to `Wav2Vec2FeatureExtractor`'s call method. When wrapping the processor into the `as_target_processor` context, however, the same method is redirected to `Wav2Vec2CTCTokenizer`'s call method.
For more information please check the [docs](https://huggingface.co/transformers/master/model_doc/wav2vec2.html#transformers.Wav2Vec2Processor.__call__).

In [ ]:
data= data.train_test_split(test_size=0.1, shuffle=True, seed=42)

In [ ]:
data

DatasetDict({
    train: Dataset({
        features: ['speech', 'sampling_rate', 'target_text'],
        num_rows: 6741
    })
    test: Dataset({
        features: ['speech', 'sampling_rate', 'target_text'],
        num_rows: 749
    })
})

In [ ]:
print(data['train'][0]["target_text"])
print(data['train'][0]["sampling_rate"])
print(type(data['train'][0]["speech"]))
print(len(data['train'][0]["speech"]))

tăng giúp tôi cái van tưới nước với 
16000
<class 'list'>
48960


In [ ]:
def prepare_dataset(batch):
    # check that all files have the correct sampling rate
    #batch ở đây là cả bộ train và test luôn chứ k phải từng dòng nha
    assert (
        len(set(batch["sampling_rate"])) == 1
    ), f"Make sure all inputs have the same sampling rate of {processor.feature_extractor.sampling_rate}."

    batch["input_values"] = processor(batch["speech"], sampling_rate=batch["sampling_rate"][0], padding="longest").input_values

    with processor.as_target_processor():
        batch["labels"] = processor(batch["target_text"]).input_ids
    return batch
#mỗi gọi lần trả về 1 tập các hàng, cuối cùng thì trả về cho mình 2 cái tập các hàng tương ứng với 2 tập train và test.

# def prepare_dataset(batch):
    # print(batch["speech"].shape)
    # print(batch["sampling_rate"])
    # print(batch["target_text"])

Let's apply the data preparation function to all examples.

In [ ]:
data.column_names["train"]

['speech', 'sampling_rate', 'target_text']

In [ ]:
data_prepared = data.map(prepare_dataset, remove_columns=data.column_names["train"], batch_size=8, batched=True)
data.cleanup_cache_files()
#https://huggingface.co/docs/datasets/v2.15.0/en/package_reference/main_classes#datasets.Dataset.map
#hàm map sẽ truyền lần lượt tập train và test vào hàm prepare_dataset, mỗi lần sẽ biến đổi các hàng trong tập dc truyền vào rồi trả về một tập bao gồm các hàng đã được chuyển đổi.

  0%|          | 0/843 [00:00<?, ?ba/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(


  0%|          | 0/94 [00:00<?, ?ba/s]

{'train': 0, 'test': 0}

In [ ]:
data_prepared

DatasetDict({
    train: Dataset({
        features: ['input_values', 'labels'],
        num_rows: 6741
    })
    test: Dataset({
        features: ['input_values', 'labels'],
        num_rows: 749
    })
})

**Note**: Currently `datasets` make use of [`torchaudio`](https://pytorch.org/audio/stable/index.html) and [`librosa`](https://librosa.org/doc/latest/index.html) for audio loading and resampling. If you wish to implement your own costumized data loading/sampling, feel free to just make use of the `"path"` column instead and disregard the `"audio"` column.

Long input sequences require a lot of memory. Since `Wav2Vec2` is based on `self-attention` the memory requirement scales quadratically with the input length for long input sequences (*cf.* with [this](https://www.reddit.com/r/MachineLearning/comments/genjvb/d_why_is_the_maximum_input_sequence_length_of/) reddit post). For this demo, let's filter all sequences that are longer than 4 seconds out of the training dataset.

Awesome, now we are ready to start training!

## Training & Evaluation

The data is processed so that we are ready to start setting up the training pipeline. We will make use of 🤗's [Trainer](https://huggingface.co/transformers/master/main_classes/trainer.html?highlight=trainer) for which we essentially need to do the following:

- Define a data collator. In contrast to most NLP models, Wav2Vec2 has a much larger input length than output length. *E.g.*, a sample of input length 50000 has an output length of no more than 100. Given the large input sizes, it is much more efficient to pad the training batches dynamically meaning that all training samples should only be padded to the longest sample in their batch and not the overall longest sample. Therefore, fine-tuning Wav2Vec2 requires a special padding data collator, which we will define below

- Evaluation metric. During training, the model should be evaluated on the word error rate. We should define a `compute_metrics` function accordingly

- Load a pretrained checkpoint. We need to load a pretrained checkpoint and configure it correctly for training.

- Define the training configuration.

After having fine-tuned the model, we will correctly evaluate it on the test data and verify that it has indeed learned to correctly transcribe speech.

### Set-up Trainer

Let's start by defining the data collator. The code for the data collator was copied from [this example](https://github.com/huggingface/transformers/blob/9a06b6b11bdfc42eea08fa91d0c737d1863c99e3/examples/research_projects/wav2vec2/run_asr.py#L81).

Without going into too many details, in contrast to the common data collators, this data collator treats the `input_values` and `labels` differently and thus applies to separate padding functions on them (again making use of Wav2Vec2's context manager). This is necessary because in speech input and output are of different modalities meaning that they should not be treated by the same padding function.
Analogous to the common data collators, the padding tokens in the labels with `-100` so that those tokens are **not** taken into account when computing the loss.

In [ ]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [ ]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

Next, the evaluation metric is defined. As mentioned earlier, the
predominant metric in ASR is the word error rate (WER), hence we will use it in this notebook as well.

In [ ]:
wer_metric = load_metric("wer")

Downloading:   0%|          | 0.00/1.90k [00:00<?, ?B/s]

The model will return a sequence of logit vectors:
$\mathbf{y}_1, \ldots, \mathbf{y}_m$ with $\mathbf{y}_1 = f_{\theta}(x_1, \ldots, x_n)[0]$ and $n >> m$.

A logit vector $\mathbf{y}_1$ contains the log-odds for each word in the vocabulary we defined earlier, thus $\text{len}(\mathbf{y}_i) =$ `config.vocab_size`. We are interested in the most likely prediction of the model and thus take the `argmax(...)` of the logits. Also, we transform the encoded labels back to the original string by replacing `-100` with the `pad_token_id` and decoding the ids while making sure that consecutive tokens are **not** grouped to the same token in CTC style ${}^1$.

In [ ]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

Now, we can load the pretrained `Wav2Vec2` checkpoint. The tokenizer's `pad_token_id` must be to define the model's `pad_token_id` or in the case of `Wav2Vec2ForCTC` also CTC's *blank token* ${}^2$. To save GPU memory, we enable PyTorch's [gradient checkpointing](https://pytorch.org/docs/stable/checkpoint.html) and also set the loss reduction to "*mean*".

In [ ]:
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
    "nguyenvulebinh/wav2vec2-base-vietnamese-250h",
    ctc_loss_reduction="mean",
    pad_token_id=processor.tokenizer.pad_token_id,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.65k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/configuration_utils.py:381: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/378M [00:00<?, ?B/s]

Some weights of the model checkpoint at nguyenvulebinh/wav2vec2-base-vietnamese-250h were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_v', 'wav2vec2.encoder.pos_conv_embed.conv.weight_g']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at nguyenvulebinh/wav2vec2-base-vietnamese-250h and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You

The first component of Wav2Vec2 consists of a stack of CNN layers that are used to extract acoustically meaningful - but contextually independent - features from the raw speech signal. This part of the model has already been sufficiently trained during pretrainind and as stated in the [paper](https://arxiv.org/abs/2006.11477) does not need to be fine-tuned anymore.
Thus, we can set the `requires_grad` to `False` for all parameters of the *feature extraction* part.

In [ ]:
model.freeze_feature_encoder()

In a final step, we define all parameters related to training.
To give more explanation on some of the parameters:
- `group_by_length` makes training more efficient by grouping training samples of similar input length into one batch. This can significantly speed up training time by heavily reducing the overall number of useless padding tokens that are passed through the model
- `learning_rate` and `weight_decay` were heuristically tuned until fine-tuning has become stable. Note that those parameters strongly depend on the Timit dataset and might be suboptimal for other speech datasets.

For more explanations on other parameters, one can take a look at the [docs](https://huggingface.co/transformers/master/main_classes/trainer.html?highlight=trainer#trainingarguments).

During training, a checkpoint will be uploaded asynchronously to the hub every 400 training steps. It allows you to also play around with the demo widget even while your model is still training.

**Note**: If one does not want to upload the model checkpoints to the hub, simply set `push_to_hub=False`.

In [ ]:
!pip install transformers[torch]

In [ ]:
!pip install accelerate -U
#update xong nhớ restart runtime, chạy lại các ô chứa biến trên RAM, thư viện với ổ đĩa k bị xóa
#để lần sau khỏi phải cứ cài bản cũ r update lên bản mới(phải restart runtime mệt) thì ta bỏ cái này lên đầu chạy cài bản mới thẳng luôn

In [ ]:
import accelerate
accelerate.__version__

'0.25.0'

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir="QuangTuan/CS337_finetuning_wav2vecbase_vietnamese_soict23slu_nguyenlevubinh250h",
  group_by_length=True,
  per_device_train_batch_size=16,
  evaluation_strategy="steps",
  num_train_epochs=2,
  fp16=True,
  gradient_checkpointing=True,
  save_steps=500,
  eval_steps=500,
  logging_steps=500,
  learning_rate=1e-4,
  weight_decay=0.005,
  warmup_steps=1000,
  save_total_limit=2,
)

Now, all instances can be passed to Trainer and we are ready to start training!

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=data_prepared["train"],
    eval_dataset=data_prepared["test"],
    tokenizer=processor.feature_extractor,
)



---

${}^1$ To allow models to become independent of the speaker rate, in CTC, consecutive tokens that are identical are simply grouped as a single token. However, the encoded labels should not be grouped when decoding since they don't correspond to the predicted tokens of the model, which is why the `group_tokens=False` parameter has to be passed. If we wouldn't pass this parameter a word like `"hello"` would incorrectly be encoded, and decoded as `"helo"`.

${}^2$ The blank token allows the model to predict a word, such as `"hello"` by forcing it to insert the blank token between the two l's. A CTC-conform prediction of `"hello"` of our model would be `[PAD] [PAD] "h" "e" "e" "l" "l" [PAD] "l" "o" "o" [PAD]`.

### Training

Training will take between 90 and 270 minutes depending on the GPU allocated to this notebook. While the trained model yields satisfying results on *Timit*'s test data, it is by no means an optimally fine-tuned model. The purpose of this notebook is to demonstrate how Wav2Vec2's [base](https://huggingface.co/facebook/wav2vec2-base), [large](https://huggingface.co/facebook/wav2vec2-large), and [large-lv60](https://huggingface.co/facebook/wav2vec2-large-lv60) checkpoints can be fine-tuned on any English dataset.

In case you want to use this google colab to fine-tune your model, you should make sure that your training doesn't stop due to inactivity. A simple hack to prevent this is to paste the following code into the console of this tab (*right mouse click -> inspect -> Console tab and insert code*).

```javascript
function ConnectButton(){
    console.log("Connect pushed");
    document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click()
}
setInterval(ConnectButton,60000);
```

Depending on what GPU was allocated to your google colab it might be possible that you are seeing an `"out-of-memory"` error here. In this case, it's probably best to reduce `per_device_train_batch_size` to 16 or even less and eventually make use of [`gradient_accumulation`](https://huggingface.co/transformers/master/main_classes/trainer.html#trainingarguments).

In [ ]:
import numpy as np
trainer.train()
#https://discuss.huggingface.co/t/fine-tune-wav2vec2-for-english-asr-with-transformers-article-bug/10440/5
#lỗi này: ValueError: Label values must be <= vocab_size: 32
# ta nghĩ ngay đến việc số kí tự trong vocab vượt quá giới hạn của pretrained model
#-> tại sao pretrained model ở đây chỉ có 32 kí tự-> dùng sai pretrained

Step,Training Loss,Validation Loss,Wer
500,2.748000,0.939024,0.541815


/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


TrainOutput(global_step=844, training_loss=2.035727008259127, metrics={'train_runtime': 1346.377, 'train_samples_per_second': 10.014, 'train_steps_per_second': 0.627, 'total_flos': 7.844159659227852e+17, 'train_loss': 2.035727008259127, 'epoch': 2.0})

The final WER should be around 0.3 which is reasonable given that state-of-the-art phoneme error rates (PER) are just below 0.1 (see [leaderboard](https://paperswithcode.com/sota/speech-recognition-on-timit)) and that WER is usually worse than PER.

You can now upload the result of the training to the Hub, just execute this instruction:

In [ ]:
trainer.push_to_hub()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


events.out.tfevents.1703759519.f82c55bae865.434.1:   0%|          | 0.00/6.01k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/378M [00:00<?, ?B/s]

events.out.tfevents.1703760330.f82c55bae865.434.2:   0%|          | 0.00/6.69k [00:00<?, ?B/s]

events.out.tfevents.1703758817.f82c55bae865.434.0:   0%|          | 0.00/6.10k [00:00<?, ?B/s]

Upload 5 LFS files:   0%|          | 0/5 [00:00<?, ?it/s]

training_args.bin:   0%|          | 0.00/4.66k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/QuangTuan/CS337_finetuning_wav2vecbase_vietnamese_soict23slu_nguyenlevubinh250h/commit/f006b409fde7dc8360e4004c287be97fd917cf7b', commit_message='End of training', commit_description='', oid='f006b409fde7dc8360e4004c287be97fd917cf7b', pr_url=None, pr_revision=None, pr_num=None)

You can now share this model with all your friends, family, favorite pets: they can all load it with the identifier "your-username/the-name-you-picked" so for instance:

```python
from transformers import AutoModelForCTC, Wav2Vec2Processor

model = AutoModelForCTC.from_pretrained("patrickvonplaten/wav2vec2-base-timit-demo-google-colab")
processor = Wav2Vec2Processor.from_pretrained("patrickvonplaten/wav2vec2-base-timit-demo-google-colab")
```

### Evaluate

In the final part, we run our model on some of the validation data to get a feeling for how well it works.

Let's load the `processor` and `model`.

In [ ]:
#có thể là do chưa push processor lên, nếu v ko cần chạy dòng này, hãy dùng processor ở trên được tạo từ tokenizer và feature extractor
processor = Wav2Vec2Processor.from_pretrained("QuangTuan/CS337_finetuning_wav2vecbase_vietnamese_soict23slu_nguyenlevubinh250h")

/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:53: FutureWarning: Loading a tokenizer inside Wav2Vec2Processor from a config that does not include a `tokenizer_class` attribute is deprecated and will be removed in v5. Please add `'tokenizer_class': 'Wav2Vec2CTCTokenizer'` attribute to either your `config.json` or `tokenizer_config.json` file to suppress this warning: 
  warnings.warn(


OSError: ignored

In [ ]:
model = Wav2Vec2ForCTC.from_pretrained("QuangTuan/wav2vec2-base-timit-demo-google-colab").cuda()

Now, we will make use of the `map(...)` function to predict the transcription of every test sample and to save the prediction in the dataset itself. We will call the resulting dictionary `"results"`.

**Note**: we evaluate the test data set with `batch_size=1` on purpose due to this [issue](https://github.com/pytorch/fairseq/issues/3227). Since padded inputs don't yield the exact same output as non-padded inputs, a better WER can be achieved by not padding the input at all.

In [ ]:
def map_to_result(batch):
  with torch.no_grad():
    input_values = torch.tensor(batch["input_values"], device="cuda").unsqueeze(0)
    logits = model(input_values).logits

  pred_ids = torch.argmax(logits, dim=-1)
  batch["pred_str"] = processor.batch_decode(pred_ids)[0]
  batch["text"] = processor.decode(batch["labels"], group_tokens=False)

  return batch

In [ ]:
results = timit["test"].map(map_to_result, remove_columns=timit["test"].column_names)

0ex [00:00, ?ex/s]

Let's compute the overall WER now.

In [ ]:
print("Test WER: {:.3f}".format(wer_metric.compute(predictions=results["pred_str"], references=results["text"])))

Test WER: 0.418


21.8% WER - not bad! Our demo model would have probably made it on the official [leaderboard](https://paperswithcode.com/sota/speech-recognition-on-timit).

Let's take a look at some predictions to see what errors are made by the model.

In [ ]:
show_random_elements(results)

,pred_str,text
0,young children hold avoid exposure to contageous deseses,young children should avoid exposure to contagious diseases
1,artofhichial inteligence is for real,artificial intelligence is for real
2,ther props were to step laters a chare and a pom fan,their props were two stepladders a chair and a palm fan
3,ifh people were more generos there wold be no ned for wealfar,if people were more generous there would be no need for welfare
4,the fish began to lep franticaly on the surfis of the smaleake,the fish began to leap frantically on the surface of the small lake
5,her ritehand egs whenever the bearometric preser changes,her right hand aches whenever the barometric pressure changes
6,only loyer es louve milonars,only lawyers love millionaires
7,the nearust sinigoud may not be withen walg in distence,the nearest synagogue may not be within walking distance
8,besketbal can be an entertaining sport,basketball can be an entertaining sport
9,she had your dark suit in greasy wash water al year,she had your dark suit in greasy wash water all year


It becomes clear that the predicted transcriptions are acoustically very similar to the target transcriptions, but often contain spelling or grammatical errors. This shouldn't be very surprising though given that we purely rely on Wav2Vec2 without making use of a language model.

Finally, to better understand how CTC works, it is worth taking a deeper look at the exact output of the model. Let's run the first test sample through the model, take the predicted ids and convert them to their corresponding tokens.

In [ ]:
model.to("cuda")

with torch.no_grad():
  logits = model(torch.tensor(timit["test"][:1]["input_values"], device="cuda")).logits

pred_ids = torch.argmax(logits, dim=-1)

# convert ids to tokens
" ".join(processor.tokenizer.convert_ids_to_tokens(pred_ids[0].tolist()))

'[PAD] [PAD] [PAD] [PAD] [PAD] [PAD] t t h e e | | b [PAD] [PAD] [PAD] [PAD] u n n g g [PAD] [PAD] o l l [PAD] [PAD] [PAD] [PAD] o [PAD] | | w [PAD] a s s s | | [PAD] p p l l [PAD] e e s s s e e n n t t l l [PAD] y y | | s [PAD] [PAD] [PAD] [PAD] [PAD] i t t [PAD] [PAD] [PAD] [PAD] u u u [PAD] [PAD] [PAD] [PAD] a t t [PAD] [PAD] e e d d | | n n e e a a r | | t h h e | s h [PAD] h [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] o o o r r r | [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]'

The output should make it a bit clearer how CTC works in practice. The model is to some extent invariant to speaking rate since it has learned to either just repeat the same token in case the speech chunk to be classified still corresponds to the same token. This makes CTC a very powerful algorithm for speech recognition since the speech file's transcription is often very much independent of its length.

I again advise the reader to take a look at [this](https://distill.pub/2017/ctc) very nice blog post to better understand CTC.